In [1]:
from os.path import dirname, join as pjoin
import scipy.io as sio
import numpy as np

In [2]:
def any(iterable):
    for element in iterable:
        if element:
            return True
    return False

In [3]:
mat_fname = pjoin('mesh', 'strip2.mat')

print(mat_fname)

mat_contents = sio.loadmat(mat_fname)

mesh\strip2.mat


In [4]:
p = mat_contents['p']
t = mat_contents['t']

In [5]:
[s1,s2] = p.shape
s1,s2

(3, 234)

In [6]:
if s1 == 2:
    print("Add third dimension")
    p = np.append(p,np.zeros((1,s2)),axis=0) 

In [7]:
p.shape

(3, 234)

In [8]:
TrianglesTotal=t.shape[1]
TrianglesTotal

244

Find areas of separate triangles

In [9]:
Area = np.zeros(TrianglesTotal)
Center = np.zeros((3,TrianglesTotal))
for m in range(TrianglesTotal):
        N = t[0:3,m]-1
        #print(N)
        Vec1 = p[:,N[0]] - p[:,N[1]]
        Vec2 = p[:,N[2]] - p[:,N[1]]
        Area[m] = np.linalg.norm(np.cross(Vec1,Vec2))/2
        Center[:,m]=1/3*sum(p[:,N],1)

In [10]:
print(Area.shape)
print(Center.shape)

(244,)
(3, 244)


Find all edge elements "Edge_" with at least two adjacent triangles

In [11]:
n = 0
Edge_l = []
TrianglePlus = []
TriangleMinus = []
for m in range(TrianglesTotal):
    N = t[0:3,m] - 1 
    for k in range(m+1,TrianglesTotal):
        M = t[0:3,k] - 1      
        a = 1 - np.array([1 if i else 0 for i in [all(N-M[0]),all(N-M[1]),all(N-M[2])]])
        if(sum(a)==2): #triangles m and k have common edge
            Edge_l.append([M[i] for i,j in enumerate(a) if j]); 
            TrianglePlus.append(m);
            TriangleMinus.append(k); 
            n = n + 1;

In [12]:
N = t[0:3,0] - 1
M = t[0:3,1] - 1
print(N)
print(M)

[  1   0 233]
[  4   1 233]


In [13]:
Edge_ = np.transpose(np.array(Edge_l))
Edge__ = np.transpose(np.array([[i[1], i[0]] for i in Edge_l]))
print(Edge_.shape)
print(Edge__.shape)

(2, 255)
(2, 255)


In [14]:
Edge_

array([[  1,   0,   4,   5, 230,   6, 228,   7,   6,   8,   7,   9,   8,
         10, 216,  11, 215,  12, 214,  13, 213,  14, 212,  15, 211,  16,
        210,  17, 209,  18, 208,  19, 207,  20, 206,  21, 205,  22, 204,
         23, 203,  24, 202,  25, 201,  26, 200,  27, 199,  28, 198,  29,
        197,  30, 196,  31, 195,  32, 194,  33, 193,  34, 192,  35, 191,
         36, 190,  37, 189,  38, 188,  39, 187,  40, 186,  41, 185,  42,
        184,  43, 183,  44, 182,  45, 181,  46, 180,  47, 179,  48, 178,
         49, 177,  50, 176,  51, 175,  52, 174,  53, 173,  54, 172,  55,
        171,  56, 170,  57, 169,  58, 168,   3,   2,  59, 167,  60, 166,
         61, 165,  62, 164,  63, 163,  64, 162,  65, 161,  66, 160,  67,
        159,  68, 158,  69, 157,  70, 156,  71, 155,  72, 154,  73, 153,
         74, 152,  75, 151,  76, 150,  77, 149,  78, 148,  79, 147,  80,
        146,  81, 145,  82, 144,  83, 143,  84, 142,  85, 141,  86, 140,
         87, 139,  88, 138,  89, 137,  90, 136,  91

In [15]:
Edge__

array([[233, 233, 233, 230,   4, 228,   5, 226, 226, 224, 224, 223, 223,
        216,   9, 215,  10, 214,  11, 213,  12, 212,  13, 211,  14, 210,
         15, 209,  16, 208,  17, 207,  18, 206,  19, 205,  20, 204,  21,
        203,  22, 202,  23, 201,  24, 200,  25, 199,  26, 198,  27, 197,
         28, 196,  29, 195,  30, 194,  31, 193,  32, 192,  33, 191,  34,
        190,  35, 189,  36, 188,  37, 187,  38, 186,  39, 185,  40, 184,
         41, 183,  42, 182,  43, 181,  44, 180,  45, 179,  46, 178,  47,
        177,  48, 176,  49, 175,  50, 174,  51, 173,  52, 172,  53, 171,
         54, 170,  55, 169,  56, 168,  57, 232, 232, 167,  58, 166,  59,
        165,  60, 164,  61, 163,  62, 162,  63, 161,  64, 160,  65, 159,
         66, 158,  67, 157,  68, 156,  69, 155,  70, 154,  71, 153,  72,
        152,  73, 151,  74, 150,  75, 149,  76, 148,  77, 147,  78, 146,
         79, 145,  80, 144,  81, 143,  82, 142,  83, 141,  84, 140,  85,
        139,  86, 138,  87, 137,  88, 136,  89, 135

In [16]:
EdgesTotal = len(Edge_l)
EdgesTotal

255

This block is only meaningful for T junctions. It leaves only two edge elements at a junction

In [17]:
Remove = []

In [18]:
for m in range(1,EdgesTotal):
    Edge_m = np.transpose(np.tile([np.array(Edge_)[:,m]],(EdgesTotal,1)))
    Edge_sup = list(Edge_  -Edge_m)
    Edge__sup = list(Edge__  -Edge_m)
    Ind1 = [0 if (Edge_sup[0][i] == 0) and (Edge_sup[1][i] == 0) else 1 for i in range(EdgesTotal)]
    Ind2 = [0 if (Edge__sup[0][i] == 0) and (Edge__sup[1][i] == 0) else 1 for i in range(EdgesTotal)]
    A_ = [i*j for i,j in zip(Ind1,Ind2)]
    A = [i for i,j in enumerate(A_) if j == 0]
    if(len(A)==3):    #three elements formally exist at a junction 
        
        Out=find(t(4,TrianglePlus(A))==t(4,TriangleMinus(A)));
        Remove=[Remove A(Out)];


SyntaxError: invalid syntax (<ipython-input-18-9f250371b6db>, line 5)

In [90]:
m = 1
Edge_m = np.transpose(np.tile([np.array(Edge_)[:,m]],(EdgesTotal,1)))
Edge_sup = list(Edge_  -Edge_m)
Edge__sup = list(Edge__  -Edge_m)
Ind1 = [0 if (Edge_sup[0][i] == 0) and (Edge_sup[1][i] == 0) else 1 for i in range(EdgesTotal)]
Ind2 = [0 if (Edge__sup[0][i] == 0) and (Edge__sup[1][i] == 0) else 1 for i in range(EdgesTotal)]

In [111]:
t

array([[  2,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,
         17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,
         30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
         43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,   4,  60,  61,  62,  63,  64,  65,  66,  67,
         68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,
         81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
         94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106,
        107, 108, 109, 110, 111, 112, 113, 114,   3, 115, 116, 117, 118,
        119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
        132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
        145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
        158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
        171, 172, 173, 174, 175, 176, 177, 178, 179